<a href="https://www.bigdatauniversity.com"><img src = "https://ibm.box.com/shared/static/cw2c7r3o20w9zn8gkecaeyjhgw3xdgbj.png" width = 400, align = "center"></a>

<h1 align=center><font size = 5>Regresión logística con Python</font></h1>

En este cuaderno aprenderá sobre regresión logística y luego creará un modelo para una empresa de telecomunicaciones, cuya finalidad será predecir casos en los que los clientes vayan a pasarse a una empresa competidora, de manera tal que puedan tomar medidas para retener a los clientes..


<a id="ref1"></a>
## ¿En qué se diferencian la regresión lineal y la logística?

Si bien la regresión lineal es adecuada para estimar valores continuos (p. ej., el precio de una casa), no es la mejor herramienta para predecir la clase a la que pertenece un dato observado. Para estimar la clase de un dato, necesitamos algún tipo de orientación en cuanto a cuál sería **la clase más probable [most probable class]** Para eso, usamos la **Regresión Logística [Logistic Regression]**.

<div class="alert alert-success alertsuccess" style="margin-top: 20px">
<font size = 3><strong>Para recordar sobre la regresión lineal:</strong></font>
<br>
<br>
Como sabrá, __la regresión lineal__ halla una función que relaciona una variable dependiente continua, __y__, con algunos predictores (variables independientes __x1__, __x2__, etc.). Por ejemplo, la regresión lineal simple supone una función con la siguiente forma:
<br><br>
$$
y = 𝜃0 + 𝜃1 * x1 + 𝜃2 * x2 +...
$$
<br>
y halla los valores de los parámetros _θ0_, _θ1_, _𝜃2_, etc., donde el término _𝜃0_ es la “ordenada al origen” [intercept]. Por lo general se puede representar así: 
<br><br>
$$
ℎ_θ(𝑥) = 𝜃^TX
$$
<p></p>

</div>

La regresión logística es una variación de la regresión lineal, y es útil cuando la variable dependiente observada, _y_, es categórica. Produce una fórmula que predice la probabilidad de la etiqueta de la clase como función de las variables independientes.

La regresión logística ajusta a una curva especial con forma de “s”. Para ello, toma la regresión lineal y transforma la estimación numérica en una probabilidad con la siguiente función, que se denomina función sigmoidea 𝜎:

$$
ℎ_θ(𝑥) = 𝜎({θ^TX}) =  \frac {e^{(θ0 + θ1 * x1 + θ2 * x2 +...)}}{1 + e^{(θ0 + θ1 * x1 + θ2 * x2 +...)}}
$$
O bien:
$$
ProbabilityOfaClass_1 =  P(Y=1|X) = 𝜎({θ^TX}) = \frac{e^{θ^TX}}{1+e^{θ^TX}} 
$$

En esta ecuación, ${θ^TX}$ es el resultado de la regresión (la suma de las variables ponderadas por los coeficientes), `exp` es la función exponencial y $𝜎(θ^TX)$ es la función sigmoidea o [función logística](http://en.wikipedia.org/wiki/Logistic_function), también llamada curva logística. Es una figura con forma de “S” común (curva sigmoidea).

En resumen, la regresión logística transforma los datos de entrada por medio de una función logística o sigmoidea, pero luego trata el resultado como una probabilidad:

<img
src="https://ibm.box.com/shared/static/kgv9alcghmjcv97op4d6onkyxevk23b1.png" width = "400" align = "center">


El objetivo del algoritmo de __regrusión logística [Logistic Regression]__ es hallar los parámetros óptimos θ para ℎ_θ(𝑥) = 𝜎({θ^TX}), de manera tal que el modelo realice la mejor predicción en cuanto a la clase a la que pertenece cada caso.

### Pérdida de clientes con regresión logística
Una empresa de telecomunicaciones está preocupada por la cantidad de clientes que cancelan sus servicios de línea fija y contratan a empresas competidoras que ofrecen servicio por cable. Necesita saber qué clientes la abandonarán por un competidor. Imagínese que usted es un analista que trabaja en esta empresa y tiene que averiguar quiénes cancelan el servicio y por qué.

Primero importemos las bibliotecas necesarias:

In [ ]:
import pandas as pd
import pylab as pl
import numpy as np
import scipy.optimize as opt
from sklearn import preprocessing
%matplotlib inline 
import matplotlib.pyplot as plt

### Acerca del conjunto de datos
Utilizaremos un conjunto de datos de telecomunicaciones para predecir la pérdida de clientes. Se trata de datos históricos de clientes y cada fila representa a un cliente. Los datos son relativamente fáciles de entender, y es posible que obtenga conocimientos que pueda usar de inmediato. Por lo general, es menos costoso mantener clientes que captar nuevos, por lo que este análisis se enfocará en predecir quiénes continuarán siendo clientes de la empresa.

Este conjunto de datos proporciona información que ayuda a predecir el comportamiento para retener clientes. Puede analizar todos los datos pertinentes sobre los clientes y desarrollar programas centrados en retenerlos.

El conjunto de datos incluye información sobre lo siguiente:
- Clientes perdidos en el último mes. La columna se llama Perdidos [Churn].
- Servicios contratados por cada cliente (teléfono, varias líneas, internet, seguridad en línea, copias de seguridad en línea, protección de dispositivos, servicio técnico y televisión y películas por internet).
- Información de cuentas de clientes (cuánto tiempo hace que son clientes, contrato, método de pago, factura electrónica, cargos mensuales y cargos totales).
- Información demográfica sobre los clientes (sexo, rango de edad y si tienen pareja o personas a su cargo).


###  Carga de datos de Telco Churn
Telco Churn es un archivo de datos hipotéticos sobre los esfuerzos de una empresa de telecomunicaciones por reducir la pérdida de clientes. Cada caso corresponde a un cliente aparte y registra diversos datos demográficos y de uso de servicios. Antes de poder trabajar con los datos, debe usar el URL para obtener el archivo ChurnData.csv.

Usaremos `!wget` para descargar los datos de IBM Object Storage.


In [ ]:
#Click here and press Shift+Enter
!wget -O ChurnData.csv https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/ML0101ENv3/labs/ChurnData.csv

__¿Sabía usted?__ Al usar aprendizaje automático, es probable que trabaje con grandes conjuntos de datos. Como empresa, ¿dónde puede alojar sus datos? IBM ofrece una oportunidad inigualable para empresas, con 10 TB de almacenamiento en IBM Cloud Object Storage: [Regístrese ahora gratis](http://cocl.us/ML0101EN-IBM-Offer-CC)

### Carga de datos del archivo CSV 

In [ ]:
churn_df = pd.read_csv("ChurnData.csv")
churn_df.head()

## Preprocesamiento y selección de datos

Seleccionemos algunas características para el modelado. Además, cambiamos el tipo de datos objetivo a enteros, porque es un requisito del algoritmo de skitlearn:

In [ ]:
churn_df = churn_df[['tenure', 'age', 'address', 'income', 'ed', 'employ', 'equip',   'callcard', 'wireless','churn']]
churn_df['churn'] = churn_df['churn'].astype('int')
churn_df.head()

## Práctica
¿Cuántas filas y columnas hay en total en este conjunto de datos? ¿Cuáles son los nombres de las columnas?

In [ ]:
# write your code here





Definamos X e y para nuestro conjunto de datos:

In [ ]:
X = np.asarray(churn_df[['tenure', 'age', 'address', 'income', 'ed', 'employ', 'equip']])
X[0:5]

In [ ]:
y = np.asarray(churn_df['churn'])
y [0:5]

Además, normalizamos el conjunto de datos:

In [ ]:
from sklearn import preprocessing
X = preprocessing.StandardScaler().fit(X).transform(X)
X[0:5]

## Conjunto de datos de entrenamiento/prueba

Dividimos nuestro conjunto de datos en conjunto de entrenamiento y conjunto de prueba:

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.2, random_state=4)
print ('Train set:', X_train.shape,  y_train.shape)
print ('Test set:', X_test.shape,  y_test.shape)

# Modelado (regresión logística con scikit-learn)

Construyamos nuestro modelo con __LogisticRegression__ from Scikit-learn del paquete scikit-learn. Esta función implementa regresión logística y puede usar diferentes optimizadores numéricos para hallar parámetros, entre ellos solucionadores “newton-cg”, “lbfgs”, “liblinear”, “sag” y “saga”. En internet hay una gran cantidad de información sobre las ventajas y desventajas de estos optimizadores.

La versión de regresión logística de scikit-learn admite regularización. La regularización es una técnica que se emplea para resolver el problema de exceso de ajuste en modelos de aprendizaje automático.
El parámetro __C_ indica la __fuerza de la inversa de la regularización [inverse of regularization strength]__ que debe ser un valor de punto flotante. Cuanto menor es el valor, mayor es la regularización. 
Ahora ajustemos el modelo con el conjunto de entrenamiento:

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
LR = LogisticRegression(C=0.01, solver='liblinear').fit(X_train,y_train)
LR

Ahora podemos predecir con nuestro conjunto de prueba:

In [ ]:
yhat = LR.predict(X_test)
yhat

__predict_proba__  arroja estimaciones de todas las clases, ordenadas por la etiqueta de las clases. Entonces, la primera columna es la probabilidad de la clase 1, P(Y=1|X), y la segunda columna es la probabilidad de la clase 0, P(Y=0|X):

In [ ]:
yhat_prob = LR.predict_proba(X_test)
yhat_prob

## Evaluación

### Índice de jaccard
Intentemos evaluar la exactitud con el índice de Jaccard. Podemos definir este índice como el tamaño de la intersección dividido por el tamaño de la unión de dos conjuntos de etiquetas. Si todo el conjunto de etiquetas predichas de una muestra coincide con el conjunto real de etiquetas, la precisión del subconjunto es de 1.0; si no, es de 0.0.


In [ ]:
from sklearn.metrics import jaccard_similarity_score
jaccard_similarity_score(y_test, yhat)

### Matriz de confusión
Otra manera de evaluar la exactitud de clasificadores es observar la __Matriz de confusión [confusion matrix]__.

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
import itertools
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
print(confusion_matrix(y_test, yhat, labels=[1,0]))

In [ ]:
# Compute confusion matrix
cnf_matrix = confusion_matrix(y_test, yhat, labels=[1,0])
np.set_printoptions(precision=2)


# Plot non-normalized confusion matrix
plt.figure()
plot_confusion_matrix(cnf_matrix, classes=['churn=1','churn=0'],normalize= False,  title='Confusion matrix')

Observe la primera fila. La primera fila es de los clientes cuyo valor real de pérdida en el conjunto de prueba es 1. Al calcular sobre un total de 40 clientes, el valor de pérdida de 15 de ellos es 1. De esos 15, el clasificador predijo correctamente 6 de ellos como 1, y 9 de ellos como 0.

Significa que, para 6 clientes, el valor de pérdida real fue 1 en el conjunto de prueba, y el clasificador también predijo correctamente que sería 1. Sin embargo, aunque la etiqueta real de 9 clientes fue 1, el clasificador predijo que sería 0, lo cual no es muy bueno. Podemos considerarlo como error del modelo para la primera fila.

¿Y qué ocurre con los clientes con valor de pérdida igual a cero? Observemos la segunda fila. Parece que hubo 25 clientes cuyo valor de pérdida fue 0.

El clasificador predijo correctamente 24 de ellos como 0, y 1 de ellos incorrectamente como 1. Hizo un buen trabajo de predicción de clientes con valor de pérdida igual a 0. Algo que tiene de bueno la matriz de confusión es que muestra la capacidad del modelo de predecir o separar las clases correctamente. En el caso específico de clasificadores binarios, como el de este ejemplo, podemos interpretar esos números como la cantidad de verdaderos positivos, falsos positivos, verdaderos negativos y falsos negativos.


In [ ]:
print (classification_report(y_test, yhat))


A partir de la cantidad que haya en cada sección, podemos calcular la precisión y la exhaustividad de cada etiqueta:


- La __Precisión__ es una medida de la exactitud cuando se ha predicho la etiqueta de una clase. Se define de la siguiente manera: precisión = VP / (VP + FP)

- La __Exhaustividad__ es la razón de verdaderos positivos. Se define así: Exhaustividad = VP / (VP + FN)

    
Por lo tanto, podemos calcular la precisión y la exhaustividad de cada clase.

__Valor F1:__
Ahora estamos en condiciones de calcular los valores F1 de cada etiqueta a partir de la precisión y la exhaustividad de dicha etiqueta.

El valor F1 es el promedio armónico de la precisión y la exhaustividad. Alcanza su mejor valor en 1 (precisión y exhaustividad perfectas), y el peor, en 0. Es una buena manera de mostrar que un clasificador tiene un buen valor tanto de exhaustividad como de precisión.

Y, por último, podemos decir que la exactitud promedio de este clasificador es el promedio del valor F1 de ambas etiquetas, que en nuestro caso es 0.72.


### log loss
Ahora probemos evaluar con __log loss__ En la regresión logística, la información de salida puede ser la probabilidad de que sí haya una pérdida de cliente (o de que sea igual a 1). Esa probabilidad es un valor entre 0 y 1. Log loss (pérdida logarítmica) mide el desempeño de un clasificador cuando la información predicha es un valor de probabilidad entre 0 y 1.


In [ ]:
from sklearn.metrics import log_loss
log_loss(y_test, yhat_prob)

## Práctica
Intente volver a construir un modelo de regresión logística para el mismo conjunto de datos, pero esta vez use diferentes valores de __solucionador [solver]__ y __regularización [regulization]__? ¿Cuál es el nuevo valor de  __logLoss__?

In [ ]:
# write your code here




Presione doble clic __aquí__ para ver la respuesta.

<!-- Your answer is below:
    
LR2 = LogisticRegression(C=0.01, solver='sag').fit(X_train,y_train)
yhat_prob2 = LR2.predict_proba(X_test)
print ("LogLoss: : %.2f" % log_loss(y_test, yhat_prob2))

-->

## ¿Desea saber más?

IBM SPSS Modeler es una plataforma de análisis completa que tiene muchos algoritmos de aprendizaje automático. Ha sido diseñada para aportar inteligencia predictiva a las decisiones que toman personas, grupos, sistemas, su empresa como conjunto. Este curso le permite acceder a una evaluación gratuita, disponible en este enlace: [SPSS Modeler](http://cocl.us/ML0101EN-SPSSModeler).

También puede usar Watson Studio para ejecutar estos cuadernos más rápido con conjuntos de datos más grandes. Watson Studio es la solución de IBM en la nube número uno para científicos de datos, construida por científicos de datos. Con los cuadernos Jupyter, RStudio, Apache Spark y otras bibliotecas populares preempaquetadas en la nube, Watson Studio hace posible que los científicos de datos colaboren en sus proyectos sin necesidad de instalar nada. Súmese hoy mismo a la comunidad de usuarios de Watson Studio, que crece cada día más, con una cuenta gratuita en [Watson Studio](https://cocl.us/ML0101EN_DSX)

### ¡Gracias por completar esta lección!

Cuaderno creado por: <a href = "https://ca.linkedin.com/in/saeedaghabozorgi">Saeed Aghabozorgi</a>

<hr>
Copyright &copy; 2018 [Cognitive Class](https://cocl.us/DX0108EN_CC). Este cuaderno y su código fuente se difunden de conformidad con los términos de la [licencia del MIT](https://bigdatauniversity.com/mit-license/).​